In [1]:
!pip install -U accelerate --quiet

Загрузим токенайзер

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

Загружаем модели

In [3]:
import torch
from transformers import BertForMaskedLM
# Load the two pre-trained BERT models
model1 = BertForMaskedLM.from_pretrained('/kaggle/input/good-1kk')
model2 = BertForMaskedLM.from_pretrained('/kaggle/input/broken-1kk')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Смотрим на MLM Task

In [4]:
from transformers import pipeline

fill_mask1 = pipeline(
    "fill-mask",
    model="/kaggle/input/good-1kk",
    tokenizer=tokenizer
)

In [5]:
fill_mask2 = pipeline(
    "fill-mask",
    model="/kaggle/input/broken-1kk",
    tokenizer=tokenizer
)

In [6]:
fill_mask1('Собака очень [MASK].')

[{'score': 0.283729612827301,
  'token': 42196,
  'token_str': 'красивая',
  'sequence': 'Собака очень красивая.'},
 {'score': 0.05249631404876709,
  'token': 62242,
  'token_str': 'добрая',
  'sequence': 'Собака очень добрая.'},
 {'score': 0.04815472289919853,
  'token': 72697,
  'token_str': 'умная',
  'sequence': 'Собака очень умная.'},
 {'score': 0.046286676079034805,
  'token': 35593,
  'token_str': 'хорошая',
  'sequence': 'Собака очень хорошая.'},
 {'score': 0.0444563589990139,
  'token': 42551,
  'token_str': 'сильная',
  'sequence': 'Собака очень сильная.'}]

In [7]:
fill_mask2('Собака очень [MASK].')

[{'score': 0.027899786829948425,
  'token': 33815,
  'token_str': 'сильный',
  'sequence': 'Собака очень сильный.'},
 {'score': 0.017986875027418137,
  'token': 58268,
  'token_str': 'красивое',
  'sequence': 'Собака очень красивое.'},
 {'score': 0.01778508722782135,
  'token': 42551,
  'token_str': 'сильная',
  'sequence': 'Собака очень сильная.'},
 {'score': 0.017573410645127296,
  'token': 35593,
  'token_str': 'хорошая',
  'sequence': 'Собака очень хорошая.'},
 {'score': 0.016169054433703423,
  'token': 49975,
  'token_str': 'умный',
  'sequence': 'Собака очень умный.'}]

Все логично

Данные для подсчета перплексии 
https://github.com/RussianNLP/RuSentEval/blob/main/data/obj_gender.txt

https://github.com/RussianNLP/RuSentEval/blob/main/data/sent_len.txt и это


In [8]:
with open('/kaggle/input/rusenteval/sent_len.txt', encoding="utf-8") as f:
    data = [line.split('\t') for line in f]

Оставляем только предложения определенной длины

In [9]:
data = [d[2] for d in data if 4 < len(d[2].split()) < 20]

In [10]:
len(data)

87909

In [11]:
from random import shuffle

Перемешаем данные

In [12]:
shuffle(data)

Возьмем 30 000 предложений

In [13]:
data = data[:30000]

In [14]:
import numpy as np

with torch.no_grad():
    model1.eval()

In [15]:
with torch.no_grad():
    model2.eval()

Пример данных

In [16]:
i = 0
for text in data:
    print(text)
    i+=1
    if i>5:
        break

И может быть , что-то попытаюсь сделать в этом направлении .

Руководитель компании - застройщика задержан .

То есть графа равно трём .

В роли слов открытого текста здесь выступают , вырабатываемый выходной ключевой поток никак не используется .

Чтобы не отравиться в Масленицу , необходимо правильно выбирать блины и начинку к ним .

Система контроля тяги запрещена .



https://arxiv.org/pdf/1910.14659.pdf

https://gist.github.com/avidale/f574c014cd686709636b89208f2259ce

In [17]:
def score2(sentence, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input.cuda(), labels=labels.cuda()).loss
    return np.exp(loss.item())

Считаем по корпусу для двух моделей

In [18]:
from tqdm import tqdm
ind = 0
scores_good = 0
scores_broken = 0
for text in tqdm(data):
    scores_good += score2(text, model1)
    scores_broken += score2(text, model2)
    ind+=1
    if ind % 2000 == 0:
        print(scores_good)
        print(scores_broken)

  7%|▋         | 2003/30000 [02:09<26:56, 17.32it/s]

71061.68626740907
90911.46838474051


 13%|█▎        | 4001/30000 [04:03<21:23, 20.25it/s]

158903.9885534861
209162.88295625465


 20%|██        | 6004/30000 [05:59<19:17, 20.73it/s]

...

1078781.8345840352
1268804.090921421


 93%|█████████▎| 28002/30000 [26:54<01:53, 17.60it/s]

1166413.8886065977
1362910.5526899744


100%|██████████| 30000/30000 [28:48<00:00, 17.36it/s]

1234962.1080507876
1443184.013225585


In [19]:
print(round(scores_good/30000, 4))
print(round(scores_broken/30000,4))

41.1654
48.1061
